In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
%matplotlib inline

In [2]:
#import data
regseason = pd.read_csv("data_folder/reg_season.csv")
fullseason = pd.read_csv("data_folder/fullseason.csv")
rs_ratings = pd.read_csv("data_folder/ratings.csv")
fs_ratings = pd.read_csv("data_folder/ratings.csv")

#add column with difference of score
regseason['Dif'] = regseason['PTS'] - regseason['PTS.1']
fullseason['Dif'] = fullseason['PTS'] - fullseason['PTS.1']

In [6]:
#performance rating of each game
def one_game (visitor, vis_rating, vis_index, 
              home, home_rating, home_index, 
              difference):
    if difference < 0:
        home_score = 1
        vis_score = 0
    else:
        home_score = 0
        vis_score = 1
        
    #Increase K to increase value of win/loss
    K = 32
    
    #transform rating
    trans_vis = (10 ** (vis_rating / 400))
    trans_home = (10 ** (home_rating / 400))
    
    #expected scores
    expected_vis = (trans_vis / (trans_vis + trans_home))
    expected_home = (trans_home / (trans_vis + trans_home))
    
    fs_ratings.Rating.iloc[vis_index] = (vis_rating + (K*(vis_score - expected_vis)))
    fs_ratings.Rating.iloc[home_index] = (home_rating + (K*(home_score - expected_home)))
    
    return fs_ratings.Rating.iloc[vis_index], fs_ratings.Rating.iloc[home_index]

In [7]:
#loop through games
def find_elos():
    count = 0
    while count < len(fullseason):
        vis_acronym = fullseason.Visitor.iloc[count]
        vis_index = fs_ratings[fs_ratings['Acronym']==vis_acronym].index.item()
        vis_rating = fs_ratings.Rating.iloc[vis_index]

        home_acronym = fullseason.Home.iloc[count]
        home_index = fs_ratings[fs_ratings['Acronym']==home_acronym].index.item()
        home_rating = fs_ratings.Rating.iloc[home_index]

        difference = fullseason.Dif.iloc[count]

        one_game(vis_acronym, vis_rating, vis_index,
                 home_acronym, home_rating, home_index,
                 difference)
        
        count += 1

In [8]:
pd.options.mode.chained_assignment = None 
find_elos()

In [9]:
#sort and round ratings
sorted_rs_ratings = rs_ratings.round().sort_values(by=["Rating"])
sorted_rs_ratings = pd.DataFrame(sorted_rs_ratings.iloc[::-1])

sorted_fs_ratings = fs_ratings.round().sort_values(by=["Rating"])
sorted_fs_ratings = pd.DataFrame(sorted_fs_ratings.iloc[::-1])

In [10]:
#import plotly for graphs
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
fig = px.bar(sorted_rs_ratings, x='Acronym', y='Rating', 
             labels={'Acronym':'Team', 'Rating':'ELO Rating'},height=600
            )

fig.layout.yaxis1.update(range=[1150, 1750], autorange=False)
fig.update_xaxes(tickangle=45, tickfont=dict(size=10))
fig.update_yaxes()
fig.show()

In [71]:
fig = go.Figure(data=[go.Table(
    columnwidth = [50,50],
    header=dict(values=[['<b> Team </b>'],['<b>Rating</b>']],
                fill_color='rgb(239, 127, 77)',
                align='left'),
    cells=dict(values=[sorted_rs_ratings.Team, sorted_rs_ratings.Rating],
               fill_color='whitesmoke',
               align='left'))
])

fig.show()

In [70]:
fig = go.Figure(data=[go.Table(
    columnwidth = [50,50],
    header=dict(values=[['<b> Team </b>'],['<b>Rating</b>']],
                fill_color='rgb(239, 127, 77)',
                align='left'),
    cells=dict(values=[sorted_fs_ratings.Team, sorted_fs_ratings.Rating],
               fill_color='whitesmoke',
               align='left'))
])

fig.show()

In [68]:
fig = go.Figure()
fig.add_trace(go.Bar(x=sorted_rs_ratings.Acronym, 
                     y=sorted_rs_ratings.Rating, 
                     marker_color='rgb(239, 127, 77)'
                          ))
fig.layout.yaxis.update(range=[1150, 1750], autorange=False)
fig.update_xaxes(tickangle=45, tickfont=dict(size=10))
fig.update_layout(yaxis=dict(
        title='Elo Rating',
        titlefont_size=16,
        tickfont_size=14,
    ), xaxis=dict(
        title='Team',
        titlefont_size=16,
        tickfont_size=12,
    ))
                  
fig.show()

In [69]:
fig = go.Figure()
fig.add_trace(go.Bar(x=sorted_fs_ratings.Acronym, 
                     y=sorted_fs_ratings.Rating, 
                     marker_color='rgb(239, 127, 77)'
                          ))
fig.layout.yaxis.update(range=[1150, 1750], autorange=False)
fig.update_xaxes(tickangle=45, tickfont=dict(size=10))
fig.update_layout(yaxis=dict(
        title='Elo Rating',
        titlefont_size=16,
        tickfont_size=14,
    ), xaxis=dict(
        title='Team',
        titlefont_size=16,
        tickfont_size=12,
    ))
                  
fig.show()

In [85]:
fig = go.Figure()
fig.add_trace(go.Bar(x=sorted_fs_ratings.Acronym, 
                     y=sorted_fs_ratings.Rating, 
                     marker_color='rgb(181, 66, 19)',
                     name = "Full Season",
                          ))
fig.add_trace(go.Bar(x=sorted_rs_ratings.Acronym, 
                     y=sorted_rs_ratings.Rating,
                     name = "Regular Season",
                     marker_color='rgb(239, 127, 77)',
                          ))
fig.layout.yaxis.update(range=[1150, 1750], autorange=False)
fig.update_xaxes(tickangle=45, tickfont=dict(size=10))
fig.update_layout(yaxis=dict(
        title='Elo Rating',
        titlefont_size=16,
        tickfont_size=14,
    ), xaxis=dict(
        title='Team',
        titlefont_size=16,
        tickfont_size=12,
    ), legend=dict(
        x=1,
        y=1,
        bgcolor='rgba(255, 255, 255, 0)',
        bordercolor='rgba(255, 255, 255, 0)'
    ),
                 )
                  
fig.show()